In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from math import sqrt
from sklearn import neighbors
from sklearn.decomposition import PCA
from sklearn import tree
from IPython.display import Image
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
import time
from joblib import parallel_backend
from multiprocessing import Pool
from statsmodels.formula.api import ols

In [2]:
df = pd.read_pickle('genomeScores.pkl')
df1 = pd.read_pickle('genomeTags.pkl')
df2 = pd.read_pickle('links.pkl')
movies = pd.read_pickle('movies.pkl')
df4 = pd.read_csv('ratings.csv')
df5 = pd.read_pickle('tags.pkl')

In [3]:
df4.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [4]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  62423 non-null  int64 
 1   title    62423 non-null  object
 2   genres   62423 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [5]:
genres=[]
for i in range(len(movies.genres)):
    for x in movies.genres[i].split('|'):
        if x not in genres:
            genres.append(x)
len(genres)
for x in genres:
    movies[x] = 0
for i in range(len(movies.genres)):
    for x in movies.genres[i].split('|'):
        movies[x][i]=1
movies.head()

/var/folders/00/9g5qbdmn74s6zq6sq2mdmm9w0000gn/T/ipykernel_20864/1209316305.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies[x][i]=1


,movieId,title,genres,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
new_df = pd.merge(df4, movies, left_on=['movieId'], right_on=['movieId'], how='inner')

new_df

,userId,movieId,rating,timestamp,title,genres,Adventure,Animation,Children,Comedy,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,3,296,5.0,1439474476,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,4,296,4.0,1573938898,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,5,296,4.0,830786155,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,7,296,4.0,835444730,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25000090,162358,200192,2.0,1553453039,Den frusna leoparden (1986),(no genres listed),0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
25000091,162358,200194,2.0,1553453843,Tough Luck (2004),Action|Adventure|Thriller,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25000092,162386,139970,3.5,1549215965,I Don't Speak English (1995),Comedy,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
25000093,162386,200726,4.0,1554651417,The Graduates (1995),Children|Drama,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
new_df = new_df.drop(['userId', 'timestamp','genres','title','movieId'], axis=1)

In [8]:
new_df

,rating,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,5.0,0,0,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,5.0,0,0,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,4.0,0,0,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,4.0,0,0,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,4.0,0,0,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25000090,2.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
25000091,2.0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
25000092,3.5,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25000093,4.0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
X = new_df.drop('rating', axis=1)

y = new_df['rating'].copy()

In [13]:
from sklearn.linear_model import LinearRegression
mod_sk = LinearRegression(fit_intercept=False).fit(X,y)

In [14]:
mod_sk_results = pd.DataFrame({'Predictor':X.columns,
                    'Coefficient':mod_sk.coef_.flatten()})

mod_sk_results.sort_values(by='Coefficient', ascending=False)

print(mod_sk_results)

             Predictor  Coefficient
0            Adventure     0.913599
1            Animation     0.445708
2             Children     0.485936
3               Comedy     1.721494
4              Fantasy     0.643203
5              Romance     0.723883
6                Drama     2.102509
7               Action     0.844542
8                Crime     0.877736
9             Thriller     0.903177
10              Horror     1.273306
11             Mystery     0.693621
12              Sci-Fi     1.031521
13                IMAX     0.417964
14         Documentary     3.221028
15                 War     0.806254
16             Musical     0.728334
17             Western     1.259654
18           Film-Noir     1.213936
19  (no genres listed)     3.326379


In [15]:
print(mod_sk.coef_)

[0.91359927 0.44570771 0.48593638 1.72149421 0.64320318 0.72388318
 2.10250856 0.84454182 0.87773619 0.90317682 1.27330644 0.69362059
 1.03152101 0.41796445 3.22102848 0.80625374 0.7283339  1.25965368
 1.2139359  3.32637924]


In [16]:
np.dot(np.dot(np.linalg.inv(np.dot(X.transpose(),
                                   X)),
              X.transpose()),
       y).flatten()

array([0.91359927, 0.44570771, 0.48593638, 1.72149421, 0.64320318,
       0.72388318, 2.10250856, 0.84454182, 0.87773619, 0.90317682,
       1.27330644, 0.69362059, 1.03152101, 0.41796445, 3.22102848,
       0.80625374, 0.7283339 , 1.25965368, 1.2139359 , 3.32637924])